# はじめてのインテリジェント・エージェントチーム構築：ADKで進化する天気ボット

このチュートリアルは [Agent Development Kit（ADK）クイックスタート例](https://google.github.io/adk-docs/get-started/quickstart/) を発展させ、より高度な **マルチエージェントシステム** の構築方法を学びます。

まずは天気情報を調べるシンプルなエージェントから始め、以下のような機能を段階的に追加していきます。

*   複数のAIモデル（Gemini, GPT, Claude）の活用
*   挨拶や別れなど、特定のタスクに特化したサブエージェントの設計
*   エージェント間のインテリジェントなタスク委譲
*   セッション状態によるエージェントの記憶機能
*   コールバックによる安全性ガードレールの実装

**なぜ天気ボットチームなのか？**

この題材はシンプルですが、実践的かつ親しみやすい例として、複雑な現実世界のエージェントアプリケーション構築に不可欠なADKのコア概念を学ぶのに最適です。インタラクションの構造化、状態管理、安全性確保、複数AI「頭脳」の協調などを体験できます。

**ADKとは？**

ADKは、LLM（大規模言語モデル）を活用したアプリケーション開発を効率化するPythonフレームワークです。エージェントの推論・計画・ツール利用・動的ユーザー対話・チーム協調などの強力な構成要素を提供します。

**この上級チュートリアルで習得できること：**

*   ✅ **ツール定義と活用**：エージェントに特定の能力（データ取得など）を与えるPython関数（ツール）の作成と、エージェントへの効果的な使い方の指示
*   ✅ **マルチLLM対応**：LiteLLM連携により、Gemini・GPT-4o・Claude Sonnetなど複数のLLMをタスクごとに使い分ける設定
*   ✅ **エージェントの委譲と協調**：専門サブエージェントの設計と、自動ルーティング（auto flow）による最適エージェントへのタスク振り分け
*   ✅ **セッション状態による記憶**：`Session State` や `ToolContext` を活用し、会話をまたいで情報を記憶・活用する方法
*   ✅ **コールバックによる安全性ガードレール**：`before_model_callback` や `before_tool_callback` を使い、事前ルールに基づくリクエストやツール利用の検査・修正・ブロック

**完成イメージ：**

このチュートリアルを終えると、天気情報の提供だけでなく、挨拶や別れの対応、最後に調べた都市の記憶、安全な動作制御まで備えたマルチエージェント天気ボットシステムをADKで構築できるようになります。

**前提条件：**

*   ✅ **Pythonプログラミングの基礎知識**
*   ✅ **LLM・API・エージェントの基本概念の理解**
*   ❗ **ADKクイックスタート等での基礎知識（Agent, Runner, SessionService, 基本的なTool利用）の習得**  
	※本チュートリアルはこれらの知識を前提としています
*   ✅ **利用予定LLMのAPIキー**（例：Google AI Studio, OpenAI Platform, Anthropic Console）

**さあ、エージェントチーム構築を始めましょう！**

In [ ]:
# @title ステップ 0: セットアップとインストール
# ADKとLiteLLMをインストールしてマルチモデルサポートを有効化

!pip install google-adk -q
!pip install litellm -q

print("インストール完了。")

In [ ]:
# @title 必要なライブラリをインポート
import os
import asyncio
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm # マルチモデルサポート用
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types # メッセージContent/Parts作成用

import warnings
# すべての警告を無視
warnings.filterwarnings("ignore")

import logging
logging.basicConfig(level=logging.ERROR)

print("ライブラリのインポート完了。")

In [ ]:
# @title APIキーを設定（実際のキーに置き換えてください！）

# --- 重要: プレースホルダーを実際のAPIキーに置き換えてください ---

# Gemini APIキー（Google AI Studioから取得: https://aistudio.google.com/app/apikey）
os.environ["GOOGLE_API_KEY"] = "YOUR_GOOGLE_API_KEY" # <--- 置き換え

# OpenAI APIキー（OpenAI Platformから取得: https://platform.openai.com/api-keys）
os.environ['OPENAI_API_KEY'] = 'YOUR_OPENAI_API_KEY' # <--- 置き換え

# Anthropic APIキー（Anthropic Consoleから取得: https://console.anthropic.com/settings/keys）
os.environ['ANTHROPIC_API_KEY'] = 'YOUR_ANTHROPIC_API_KEY' # <--- 置き換え


# --- キーの確認（オプション） ---
print("APIキー設定済み:")
print(f"Google APIキー設定済み: {'はい' if os.environ.get('GOOGLE_API_KEY') and os.environ['GOOGLE_API_KEY'] != 'YOUR_GOOGLE_API_KEY' else 'いいえ（プレースホルダーを置き換えてください！）'}")
print(f"OpenAI APIキー設定済み: {'はい' if os.environ.get('OPENAI_API_KEY') and os.environ['OPENAI_API_KEY'] != 'YOUR_OPENAI_API_KEY' else 'いいえ（プレースホルダーを置き換えてください！）'}")
print(f"Anthropic APIキー設定済み: {'はい' if os.environ.get('ANTHROPIC_API_KEY') and os.environ['ANTHROPIC_API_KEY'] != 'YOUR_ANTHROPIC_API_KEY' else 'いいえ（プレースホルダーを置き換えてください！）'}")

# ADKがAPIキーを直接使用するように設定（このマルチモデルセットアップではVertex AIを使用しない）
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "False"


# @markdown **セキュリティ注意:** APIキーを直接ハードコーディングするのではなく、Colab Secretsや環境変数を使用して安全に管理するのがベストプラクティスです。上記のプレースホルダー文字列を置き換えてください。

In [ ]:
# --- モデル定数を定義して簡単に使用できるようにする ---

MODEL_GEMINI_2_0_FLASH = "gemini-2.0-flash"

# 注意: 特定のモデル名は変更される可能性があります。LiteLLM/Providerのドキュメントを参照してください。
MODEL_GPT_4O = "openai/gpt-4o"
MODEL_CLAUDE_SONNET = "anthropic/claude-3-sonnet-20240229"


print("\n環境設定完了。")

---

## ステップ 1: 最初のエージェント \- 基本的な天気情報の取得

まずは、Weather Bot の基本コンポーネントを構築します。これは、特定のタスク（天気情報の取得）を実行できる単一のエージェントを作成することから始まります。このステップでは、以下の2つの主要な要素を作成します：

1. **ツール:** 天気データを取得する「能力」をエージェントに与える Python 関数。  
2. **エージェント:** ユーザーのリクエストを理解し、天気ツールを持っていることを認識し、それをいつどのように使用するかを判断する AI の「頭脳」。

---

**1\. ツールの定義 (`get_weather`)**

ADK において、**ツール**はエージェントにテキスト生成以外の具体的な能力を与える構成要素です。通常、ツールは特定のアクション（API 呼び出し、データベースクエリ、計算など）を実行するための通常の Python 関数です。

最初のツールでは、*モック*の天気レポートを提供します。これにより、外部 API キーを必要とせずにエージェントの構造に集中できます。後で、このモック関数を実際の天気サービスを呼び出すものに簡単に置き換えることができます。

**重要な概念: Docstring が重要\!** エージェントの LLM は、関数の **docstring** に大きく依存して以下を理解します：

* ツールが「何をする」のか。  
* ツールを「いつ使用するべき」なのか。  
* ツールが必要とする「引数」（例: `city: str`）。  
* ツールが返す「情報」。

**ベストプラクティス:** ツールの docstring を明確で正確に記述してください。これは、LLM がツールを正しく使用するために不可欠です。

In [ ]:
# @title 天気ツールを定義する
def get_weather(city: str) -> dict:
    """指定された都市の現在の天気レポートを取得します。

    Args:
        city (str): 都市名（例: "New York", "London", "Tokyo"）。

    Returns:
        dict: 天気情報を含む辞書。
              'status'キー（'success'または'error'）を含みます。
              'success'の場合、'report'キーに天気の詳細が含まれます。
              'error'の場合、'error_message'キーが含まれます。
    """
    print(f"--- ツール: get_weather が呼び出されました。都市: {city} ---") # ツール実行ログ
    city_normalized = city.lower().replace(" ", "") # 基本的な正規化

    # モック天気データ
    mock_weather_db = {
        "newyork": {"status": "success", "report": "ニューヨークの天気は晴れで、気温は25°Cです。"},
        "london": {"status": "success", "report": "ロンドンは曇りで、気温は15°Cです。"},
        "tokyo": {"status": "success", "report": "東京は小雨が降っており、気温は18°Cです。"},
    }

    if city_normalized in mock_weather_db:
        return mock_weather_db[city_normalized]
    else:
        return {"status": "error", "error_message": f"申し訳ありませんが、「{city}」の天気情報はありません。"}

# ツール使用例（オプションのテスト）
print(get_weather("New York"))
print(get_weather("Paris"))

---

**2\. エージェントの定義 (`weather_agent`)**

次に、**エージェント**自体を作成します。ADKにおける`Agent`は、ユーザー、LLM、および利用可能なツール間のインタラクションを調整する役割を果たします。

以下の主要なパラメータを設定します：

* `name`: このエージェントの一意の識別子（例: "weather\_agent\_v1"）。  
* `model`: 使用するLLMを指定（例: `MODEL_GEMINI_2_5_PRO`）。ここでは、特定のGeminiモデルから始めます。  
* `description`: エージェントの全体的な目的を簡潔にまとめた説明。この説明は、他のエージェントがタスクを*この*エージェントに委譲するかどうかを判断する際に重要になります。  
* `instruction`: LLMに対する詳細な指示。エージェントの役割、目標、割り当てられた`tools`を*どのように、いつ*使用するかを具体的に説明します。  
* `tools`: エージェントが使用を許可されているPythonツール関数のリスト（例: `[get_weather]`）。

**ベストプラクティス:** 明確で具体的な`instruction`プロンプトを提供してください。指示が詳細であるほど、LLMはその役割やツールの使用方法をよりよく理解できます。エラー処理についても必要に応じて明示してください。

**ベストプラクティス:** 説明的な`name`と`description`の値を選択してください。これらはADK内部で使用され、自動委譲（後述）のような機能において重要です。

In [ ]:
# @title 天気エージェントを定義する
# 以前に定義したモデル定数を使用
AGENT_MODEL = MODEL_GEMINI_2_0_FLASH # Geminiから始める

weather_agent = Agent(
    name="weather_agent_v1",
    model=AGENT_MODEL, # GeminiまたはLiteLlmオブジェクトの文字列を指定
    description="特定の都市の天気情報を提供します。",
    instruction="あなたは役立つ天気アシスタントです。 "
                "ユーザーが特定の都市の天気を尋ねたとき、 "
                "'get_weather'ツールを使用して情報を見つけてください。 "
                "ツールがエラーを返した場合、丁寧にユーザーに伝えてください。 "
                "ツールが成功した場合、天気レポートを明確に提示してください。",
    tools=[get_weather], # 関数を直接渡す
)

print(f"エージェント '{weather_agent.name}' がモデル '{AGENT_MODEL}' を使用して作成されました。")

---

**3\. ランナーとセッションサービスのセットアップ**

エージェントとの会話を管理し、実行するために、以下の2つのコンポーネントが必要です：

* `SessionService`（セッションサービス）：異なるユーザーやセッションの会話履歴と状態を管理します。`InMemorySessionService`は、すべてをメモリ内に保存するシンプルな実装で、テストや簡単なアプリケーションに適しています。交換されたメッセージを追跡します。ステートの永続化については、ステップ4でさらに詳しく説明します。  
* `Runner`（ランナー）：インタラクションフローを調整するエンジンです。ユーザー入力を受け取り、適切なエージェントにルーティングし、エージェントのロジックに基づいてLLMやツールへの呼び出しを管理し、`SessionService`を介してセッションを更新し、インタラクションの進行状況を表すイベントを生成します。

In [ ]:
# @title セッションサービスとランナーのセットアップ

# --- セッション管理 ---
# キーコンセプト: SessionServiceは会話履歴と状態を保存します。
# InMemorySessionServiceは、このチュートリアルのためのシンプルで永続化されないストレージです。
session_service = InMemorySessionService()

# インタラクションコンテキストを識別するための定数を定義
APP_NAME = "weather_tutorial_app"
USER_ID = "user_1"
SESSION_ID = "session_001" # シンプルさのために固定IDを使用

# 会話が行われる特定のセッションを作成
session = session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)
print(f"セッション作成: App='{APP_NAME}', User='{USER_ID}', Session='{SESSION_ID}'")

# --- ランナー ---
# キーコンセプト: Runnerはエージェント実行ループを調整します。
runner = Runner(
    agent=weather_agent, # 実行したいエージェント
    app_name=APP_NAME,   # 実行をアプリに関連付ける
    session_service=session_service # セッションマネージャーを使用
)
print(f"ランナーがエージェント '{runner.agent.name}' のために作成されました。")

---

**4\. エージェントとの対話**

エージェントにメッセージを送信し、その応答を受け取る方法が必要です。LLMの呼び出しやツールの実行には時間がかかる場合があるため、ADKの`Runner`は非同期で動作します。

ここでは、`async`ヘルパー関数（`call_agent_async`）を定義します。この関数は以下を行います：

1. ユーザーのクエリ文字列を受け取ります。  
2. それをADKの`Content`形式にパッケージ化します。  
3. ユーザー/セッションのコンテキストと新しいメッセージを提供して`runner.run_async`を呼び出します。  
4. ランナーが生成する**イベント（Events）**を反復処理します。イベントは、エージェントの実行ステップ（例: ツール呼び出しのリクエスト、ツール結果の受信、中間的なLLMの思考、最終応答）を表します。  
5. `event.is_final_response()`を使用して**最終応答**イベントを特定し、出力します。

**なぜ`async`を使うのか？** LLMとの対話やツール（外部APIなど）の操作はI/Oバウンドの操作です。`asyncio`を使用することで、これらの操作を効率的に処理し、実行をブロックすることなくプログラムを進行させることができます。

In [ ]:
# @title エージェント対話関数を定義

from google.genai import types # メッセージContent/Parts作成用

async def call_agent_async(query: str, runner, user_id, session_id):
  """クエリをエージェントに送信し、最終応答を出力します。"""
  print(f"\n>>> ユーザークエリ: {query}")

  # ユーザーのメッセージをADK形式で準備
  content = types.Content(role='user', parts=[types.Part(text=query)])

  final_response_text = "エージェントは最終応答を生成しませんでした。" # デフォルト

  # キーコンセプト: run_asyncはエージェントロジックを実行し、イベントを生成します。
  # イベントを反復処理して最終応答を見つけます。
  async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
      # すべてのイベントを表示するには、以下の行のコメントを解除してください
      # print(f"  [イベント] 作成者: {event.author}, タイプ: {type(event).__name__}, 最終: {event.is_final_response()}, コンテンツ: {event.content}")

      # キーコンセプト: is_final_response()はターンの最終メッセージを示します。
      if event.is_final_response():
          if event.content and event.content.parts:
             # テキスト応答を最初のパートに仮定
             final_response_text = event.content.parts[0].text
          elif event.actions and event.actions.escalate: # 潜在的なエラー/エスカレーションを処理
             final_response_text = f"エージェントがエスカレートしました: {event.error_message or '特定のメッセージはありません。'}"
          # 必要に応じて、ここにさらにチェックを追加（例: 特定のエラーコード）
          break # 最終応答が見つかったらイベント処理を停止

  print(f"<<< エージェント応答: {final_response_text}")

---

**5\. 会話を実行する**

最後に、エージェントにいくつかのクエリを送信してセットアップをテストしましょう。`async` 呼び出しをメインの `async` 関数でラップし、それを `await` を使って実行します。

出力を確認してください：

* ユーザーのクエリを確認します。  
* エージェントがツールを使用する際に表示される `--- Tool: get_weather called... ---` ログに注目してください。  
* エージェントの最終的な応答を観察し、特に天気データが利用できない場合（例: パリ）の処理方法を確認してください。

In [ ]:
# @title 初期会話を実行する

# 非同期関数を使用して対話ヘルパーを待機
async def run_conversation():
    await call_agent_async("ロンドンの天気はどうですか？",
                                       runner=runner,
                                       user_id=USER_ID,
                                       session_id=SESSION_ID)

    await call_agent_async("パリはどうですか？",
                                       runner=runner,
                                       user_id=USER_ID,
                                       session_id=SESSION_ID) # ツールのエラーメッセージを期待

    await call_agent_async("ニューヨークの天気を教えてください",
                                       runner=runner,
                                       user_id=USER_ID,
                                       session_id=SESSION_ID)

# awaitを使用して会話を実行（Colab/Jupyterのような非同期コンテキストで）
await run_conversation()

---

おめでとうございます！最初のADKエージェントを構築し、対話することに成功しました。このエージェントはユーザーのリクエストを理解し、ツールを使用して情報を取得し、その結果に基づいて適切に応答します。

次のステップでは、このエージェントを支える基盤となる言語モデルを簡単に切り替える方法を探ります。

## ステップ 2: LiteLLMを使ったマルチモデル対応

ステップ1では、特定のGeminiモデルを使用した機能的なWeather Agentを構築しました。効果的ではありますが、実際のアプリケーションでは*異なる*大規模言語モデル（LLM）を使用する柔軟性が求められることがよくあります。なぜでしょうか？

* **パフォーマンス:** 一部のモデルは特定のタスク（例: コーディング、推論、クリエイティブライティング）に優れています。
* **コスト:** モデルごとに価格が異なります。
* **機能:** モデルは多様な機能、コンテキストウィンドウサイズ、ファインチューニングオプションを提供します。
* **可用性/冗長性:** 代替手段を持つことで、1つのプロバイダーに問題が発生してもアプリケーションが機能し続けることが保証されます。

ADKは、[**LiteLLM**](https://github.com/BerriAI/litellm)ライブラリとの統合を通じて、モデルの切り替えをシームレスに行います。LiteLLMは、100以上の異なるLLMに対する一貫したインターフェースとして機能します。

**このステップでは、以下を行います：**

1. OpenAI（GPT）やAnthropic（Claude）などのプロバイダーからモデルを使用するようにADK `Agent`を設定する方法を学びます。
2. Weather Agentのインスタンスを定義し、それぞれ異なるLLMをバックエンドに持つように設定し（独自のセッションとランナーを持つ）、すぐにテストします。
3. これらの異なるエージェントと対話し、同じ基礎ツールを使用しても応答にどのような違いがあるかを観察します。

---

**1\. `LiteLlm`のインポート**

初期セットアップ（ステップ0）でこれをインポートしましたが、マルチモデルサポートのための重要なコンポーネントです：

In [ ]:
# @title 1. LiteLlmをインポート
from google.adk.models.lite_llm import LiteLlm

**2\. マルチモデルエージェントの定義とテスト**

特定のGeminiモデルの文字列名を渡す代わりに、`LiteLlm`クラス内に希望するモデル識別子文字列をラップします。

* **キーコンセプト: `LiteLlm`ラッパー:** `LiteLlm(model="provider/model_name")`構文は、ADKにこのエージェントのリクエストをLiteLLMライブラリを通じて指定されたモデルプロバイダーにルーティングするよう指示します。

ステップ0で必要なAPIキーが設定されていることを確認してください。`call_agent_async`関数（以前に定義され、現在は`runner`、`user_id`、`session_id`を受け入れる）を使用して、各エージェントのセットアップ直後に対話します。

以下の各ブロックは：

* 特定のLiteLLMモデル（`MODEL_GPT_4O`または`MODEL_CLAUDE_SONNET`）を使用してエージェントを定義します。
* そのエージェントのテスト実行専用の*新しい、別の* `InMemorySessionService`とセッションを作成します。これにより、会話履歴が分離されます。
* 特定のエージェントとそのセッションサービスに設定された`Runner`を作成します。
* すぐに`call_agent_async`を呼び出してクエリを送信し、エージェントをテストします。

**ベストプラクティス:** モデル名の定数（`MODEL_GPT_4O`、`MODEL_CLAUDE_SONNET`など）を使用して、タイプミスを防ぎ、コードを管理しやすくします。

**エラーハンドリング:** エージェント定義を`try...except`ブロックでラップします。これにより、特定のプロバイダーのAPIキーが欠落しているか無効である場合でも、チュートリアル全体が失敗するのを防ぎ、設定されているモデルで続行できるようにします。

まず、OpenAIのGPT-4oを使用するエージェントを作成してテストしましょう。

In [ ]:
# @title GPTエージェントを定義してテストする

# 'get_weather'関数が定義されていることを確認（ステップ1から）。
# 'call_agent_async'が以前に定義されていることを確認。

# --- GPT-4oを使用するエージェント ---
weather_agent_gpt = None # Noneに初期化
runner_gpt = None      # ランナーをNoneに初期化

try:
    weather_agent_gpt = Agent(
        name="weather_agent_gpt",
        # 重要な変更: LiteLLMモデル識別子をラップ
        model=LiteLlm(model=MODEL_GPT_4O),
        description="天気情報を提供します（GPT-4oを使用）。",
        instruction="あなたはGPT-4oを搭載した役立つ天気アシスタントです。 "
                    "都市の天気リクエストには 'get_weather' ツールを使用してください。 "
                    "ツールの出力ステータスに基づいて、成功したレポートや丁寧なエラーメッセージを明確に提示してください。",
        tools=[get_weather], # 同じツールを再利用
    )
    print(f"エージェント '{weather_agent_gpt.name}' がモデル '{MODEL_GPT_4O}' を使用して作成されました。")

    # InMemorySessionServiceは、このチュートリアルのためのシンプルで永続化されないストレージです。
    session_service_gpt = InMemorySessionService() # 専用サービスを作成

    # インタラクションコンテキストを識別するための定数を定義
    APP_NAME_GPT = "weather_tutorial_app_gpt" # このテストのための一意のアプリ名
    USER_ID_GPT = "user_1_gpt"
    SESSION_ID_GPT = "session_001_gpt" # シンプルさのために固定IDを使用

    # 会話が行われる特定のセッションを作成
    session_gpt = session_service_gpt.create_session(
        app_name=APP_NAME_GPT,
        user_id=USER_ID_GPT,
        session_id=SESSION_ID_GPT
    )
    print(f"セッション作成: App='{APP_NAME_GPT}', User='{USER_ID_GPT}', Session='{SESSION_ID_GPT}'")

    # このエージェントとそのセッションサービスに特化したランナーを作成
    runner_gpt = Runner(
        agent=weather_agent_gpt,
        app_name=APP_NAME_GPT,       # 特定のアプリ名を使用
        session_service=session_service_gpt # 特定のセッションサービスを使用
        )
    print(f"ランナーがエージェント '{runner_gpt.agent.name}' のために作成されました。")

    # --- GPTエージェントをテスト ---
    print("\n--- GPTエージェントをテスト ---")
    # call_agent_asyncが正しいランナー、user_id、session_idを使用することを確認
    await call_agent_async(query = "東京の天気はどうですか？",
                           runner=runner_gpt,
                           user_id=USER_ID_GPT,
                           session_id=SESSION_ID_GPT)

except Exception as e:
    print(f"❌ GPTエージェント '{MODEL_GPT_4O}' を作成または実行できませんでした。APIキーとモデル名を確認してください。エラー: {e}")


次に、AnthropicのClaude Sonnetを使用して同じことを行います。

In [ ]:
# @title Claudeエージェントを定義してテストする

# 'get_weather'関数が定義されていることを確認（ステップ1から）。
# 'call_agent_async'が以前に定義されていることを確認。

# --- Claude Sonnetを使用するエージェント ---
weather_agent_claude = None # Noneに初期化
runner_claude = None      # ランナーをNoneに初期化

try:
    weather_agent_claude = Agent(
        name="weather_agent_claude",
        # 重要な変更: LiteLLMモデル識別子をラップ
        model=LiteLlm(model=MODEL_CLAUDE_SONNET),
        description="天気情報を提供します（Claude Sonnetを使用）。",
        instruction="あなたはClaude Sonnetを搭載した役立つ天気アシスタントです。 "
                    "都市の天気リクエストには 'get_weather' ツールを使用してください。 "
                    "ツールの辞書出力（'status'、'report'/'error_message'）を分析してください。 "
                    "成功したレポートや丁寧なエラーメッセージを明確に提示してください。",
        tools=[get_weather], # 同じツールを再利用
    )
    print(f"エージェント '{weather_agent_claude.name}' がモデル '{MODEL_CLAUDE_SONNET}' を使用して作成されました。")

    # InMemorySessionServiceは、このチュートリアルのためのシンプルで永続化されないストレージです。
    session_service_claude = InMemorySessionService() # 専用サービスを作成

    # インタラクションコンテキストを識別するための定数を定義
    APP_NAME_CLAUDE = "weather_tutorial_app_claude" # 一意のアプリ名
    USER_ID_CLAUDE = "user_1_claude"
    SESSION_ID_CLAUDE = "session_001_claude" # シンプルさのために固定IDを使用

    # 会話が行われる特定のセッションを作成
    session_claude = session_service_claude.create_session(
        app_name=APP_NAME_CLAUDE,
        user_id=USER_ID_CLAUDE,
        session_id=SESSION_ID_CLAUDE
    )
    print(f"セッション作成: App='{APP_NAME_CLAUDE}', User='{USER_ID_CLAUDE}', Session='{SESSION_ID_CLAUDE}'")

    # このエージェントとそのセッションサービスに特化したランナーを作成
    runner_claude = Runner(
        agent=weather_agent_claude,
        app_name=APP_NAME_CLAUDE,       # 特定のアプリ名を使用
        session_service=session_service_claude # 特定のセッションサービスを使用
        )
    print(f"ランナーがエージェント '{runner_claude.agent.name}' のために作成されました。")

    # --- Claudeエージェントをテスト ---
    print("\n--- Claudeエージェントをテスト ---")
    # call_agent_asyncが正しいランナー、user_id、session_idを使用することを確認
    await call_agent_async(query = "ロンドンの天気を教えてください。",
                           runner=runner_claude,
                           user_id=USER_ID_CLAUDE,
                           session_id=SESSION_ID_CLAUDE)

except Exception as e:
    print(f"❌ Claudeエージェント '{MODEL_CLAUDE_SONNET}' を作成または実行できませんでした。APIキーとモデル名を確認してください。エラー: {e}")

出力を注意深く観察してください。次のことが確認できます：

1. 各エージェント（`weather_agent_gpt`、`weather_agent_claude`）が正常に作成される（APIキーが有効である場合）。
2. 各エージェントに専用のセッションとランナーが設定される。
3. 各エージェントがクエリを処理する際に`get_weather`ツールを使用する必要があることを正しく認識する（`--- Tool: get_weather called... ---`ログが表示される）。
4. **基礎ツールロジック**は同一であり、常にモックデータを返す。
5. しかし、**最終的なテキスト応答**は、各エージェントによって生成されるため、表現、トーン、フォーマットに若干の違いがあるかもしれません。これは、指示プロンプトが異なるLLM（GPT-4o対Claude Sonnet）によって解釈され、実行されるためです。

このステップは、ADK + LiteLLMが提供するパワーと柔軟性を示しています。コアアプリケーションロジック（ツール、基本的なエージェント構造）を一貫して保ちながら、さまざまなLLMを使用してエージェントを簡単に実験し、展開できます。

次のステップでは、単一のエージェントを超えて、エージェントが互いにタスクを委譲できる小さなチームを構築します！

---

## ステップ 3: エージェントチームの構築 \- 挨拶と別れの委譲

ステップ1と2では、天気の検索に特化した単一のエージェントを構築し、実験しました。特定のタスクには効果的ですが、実際のアプリケーションでは、より多様なユーザーインタラクションを処理する必要があります。*1つの天気エージェントにさらに多くのツールと複雑な指示を追加することもできますが、これはすぐに管理が難しくなり、効率が低下します。*

より堅牢なアプローチは、**エージェントチーム**を構築することです。これには以下が含まれます：

1. **専門化されたエージェント**を複数作成し、それぞれが特定の能力（例: 天気、挨拶、計算）に特化します。  
2. **ルートエージェント**（またはオーケストレーター）を指定し、最初のユーザーリクエストを受け取ります。  
3. ルートエージェントがユーザーの意図に基づいて最も適切な専門サブエージェントに**タスクを委譲**できるようにします。

**なぜエージェントチームを構築するのか？**

* **モジュール性:** 個々のエージェントを開発、テスト、維持しやすくなります。  
* **専門化:** 各エージェントはその特定のタスクに合わせて（指示、モデル選択）微調整できます。  
* **スケーラビリティ:** 新しい能力を追加するのが簡単になります。新しいエージェントを追加するだけです。  
* **効率:** より簡単なタスクには、よりシンプルで安価なモデルを使用できます（例: 挨拶）。

**このステップでは、以下を行います：**

1. 挨拶（`say_hello`）と別れ（`say_goodbye`）を処理するシンプルなツールを定義します。  
2. 2つの新しい専門サブエージェントを作成します：`greeting_agent`と`farewell_agent`。  
3. メインの天気エージェント（`weather_agent_v2`）を**ルートエージェント**として更新します。  
4. ルートエージェントをサブエージェントと共に設定し、**自動委譲**を有効にします。  
5. ルートエージェントに異なるタイプのリクエストを送信して委譲フローをテストします。

---

**1\. サブエージェント用のツールを定義**

まず、エージェントの新しい専門ツールとして機能するシンプルなPython関数を作成します。明確なdocstringが重要です。

In [ ]:
# @title 挨拶と別れのエージェント用ツールを定義

# 'get_weather'が環境に定義されていることを確認（ステップ1から）。
# def get_weather(city: str) -> dict: ...（ステップ1から）

def say_hello(name: str = "there") -> str:
    """シンプルな挨拶を提供し、オプションでユーザーの名前を呼びかけます。

    Args:
        name (str, optional): 挨拶する人の名前。デフォルトは "there"。

    Returns:
        str: フレンドリーな挨拶メッセージ。
    """
    print(f"--- ツール: say_hello が呼び出されました。名前: {name} ---")
    return f"こんにちは、{name}さん！"

def say_goodbye() -> str:
    """会話を終了するためのシンプルな別れのメッセージを提供します。"""
    print(f"--- ツール: say_goodbye が呼び出されました ---")
    return "さようなら！良い一日を。"

print("挨拶と別れのツールが定義されました。")

# オプションの自己テスト
print(say_hello("Alice"))
print(say_goodbye())

---

**2\. サブエージェント（挨拶と別れ）を定義**

次に、**エージェント**自体を作成します。ADKにおける`Agent`は、ユーザー、LLM、および利用可能なツール間のインタラクションを調整する役割を果たします。

以下の主要なパラメータを設定します：

* `name`: このエージェントの一意の識別子（例: "weather\_agent\_v1"）。  
* `model`: 使用するLLMを指定（例: `MODEL_GEMINI_2_5_PRO`）。ここでは、特定のGeminiモデルから始めます。  
* `description`: エージェントの全体的な目的を簡潔にまとめた説明。この説明は、他のエージェントがタスクを*この*エージェントに委譲するかどうかを判断する際に重要になります。  
* `instruction`: LLMに対する詳細な指示。エージェントの役割、目標、割り当てられた`tools`を*どのように、いつ*使用するかを具体的に説明します。  
* `tools`: エージェントが使用を許可されているPythonツール関数のリスト（例: `[get_weather]`）。

**ベストプラクティス:** 明確で具体的な`instruction`プロンプトを提供してください。指示が詳細であるほど、LLMはその役割やツールの使用方法をよりよく理解できます。エラー処理についても必要に応じて明示してください。

**ベストプラクティス:** 説明的な`name`と`description`の値を選択してください。これらはADK内部で使用され、自動委譲（後述）のような機能において重要です。

In [ ]:
# @title 挨拶と別れのサブエージェントを定義

# 必要なインポートを確認し、APIキーが設定されていることを確認（ステップ0/2から）
# from google.adk.models.lite_llm import LiteLlm
# MODEL_GPT_4O, MODEL_CLAUDE_SONNETなどが定義されていることを確認

# --- 挨拶エージェント ---
greeting_agent = None
try:
    greeting_agent = Agent(
        # シンプルなタスクには異なる/安価なモデルを使用する可能性があります
        model=LiteLlm(model=MODEL_GPT_4O),
        name="greeting_agent",
        instruction="あなたは挨拶エージェントです。あなたの唯一のタスクは、ユーザーにフレンドリーな挨拶を提供することです。 "
                    "'say_hello'ツールを使用して挨拶を生成してください。 "
                    "ユーザーが名前を提供した場合、それをツールに渡してください。 "
                    "他の会話やタスクには関与しないでください。",
        description="シンプルな挨拶とハローを 'say_hello' ツールを使用して処理します。", # 委譲に重要
        tools=[say_hello],
    )
    print(f"✅ エージェント '{greeting_agent.name}' がモデル '{MODEL_GPT_4O}' を使用して作成されました。")
except Exception as e:
    print(f"❌ 挨拶エージェントを作成できませんでした。APIキーを確認してください（{MODEL_GPT_4O}）。エラー: {e}")

# --- 別れエージェント ---
farewell_agent = None
try:
    farewell_agent = Agent(
        # 同じまたは異なるモデルを使用できます
        model=LiteLlm(model=MODEL_GPT_4O), # この例ではGPTを使用
        name="farewell_agent",
        instruction="あなたは別れエージェントです。あなたの唯一のタスクは、丁寧な別れのメッセージを提供することです。 "
                    "ユーザーが別れや会話の終了を示す場合（例: 'bye', 'goodbye', 'thanks bye', 'see you'）、'say_goodbye'ツールを使用してください。 "
                    "他のアクションは行わないでください。",
        description="シンプルな別れとさようならを 'say_goodbye' ツールを使用して処理します。", # 委譲に重要
        tools=[say_goodbye],
    )
    print(f"✅ エージェント '{farewell_agent.name}' がモデル '{MODEL_GPT_4O}' を使用して作成されました。")
except Exception as e:
    print(f"❌ 別れエージェントを作成できませんでした。APIキーを確認してください（{MODEL_GPT_4O}）。エラー: {e}")

---

**3\. サブエージェントを持つルートエージェント（Weather Agent v2）を定義**

次に、`weather_agent`をアップグレードします。主な変更点は次のとおりです：

* `sub_agents`パラメータを追加：先ほど作成した`greeting_agent`と`farewell_agent`インスタンスを含むリストを渡します。  
* `instruction`を更新：ルートエージェントにサブエージェントについて明示的に指示し、タスクを委譲するタイミングを説明します。

**キーコンセプト: 自動委譲（Auto Flow）** `sub_agents`リストを提供することで、ADKは自動委譲を有効にします。ルートエージェントがユーザーのクエリを受け取ると、そのLLMは自身の指示とツールだけでなく、各サブエージェントの`description`も考慮します。LLMがクエリがサブエージェントの記述された能力（例: "シンプルな挨拶を処理する"）により適していると判断した場合、そのターンの制御をそのサブエージェントに*移譲*する特別な内部アクションを自動的に生成します。サブエージェントはその後、自身のモデル、指示、ツールを使用してクエリを処理します。

**ベストプラクティス:** ルートエージェントの指示が委譲の決定を明確にガイドするようにしてください。サブエージェントの名前を明記し、委譲すべき条件を説明します。

In [ ]:
# @title サブエージェントを持つルートエージェントを定義

# サブエージェントが正常に作成されたことを確認してからルートエージェントを定義します。
# また、元の 'get_weather' ツールが定義されていることを確認します。
root_agent = None
runner_root = None # ランナーを初期化

if greeting_agent and farewell_agent and 'get_weather' in globals():
    # ルートエージェントには強力なGeminiモデルを使用してオーケストレーションを行います
    root_agent_model = MODEL_GEMINI_2_0_FLASH

    weather_agent_team = Agent(
        name="weather_agent_v2", # 新しいバージョン名を付ける
        model=root_agent_model,
        description="メインのコーディネーターエージェント。天気リクエストを処理し、挨拶/別れを専門家に委譲します。",
        instruction="あなたはエージェントチームを調整するメインの天気エージェントです。主な責任は天気情報を提供することです。 "
                    "特定の天気リクエストには 'get_weather' ツールのみを使用してください（例: 'ロンドンの天気'）。 "
                    "専門のサブエージェントがいます： "
                    "1. 'greeting_agent': 'Hi', 'Hello'のようなシンプルな挨拶を処理します。これに委譲してください。 "
                    "2. 'farewell_agent': 'Bye', 'See you'のようなシンプルな別れを処理します。これに委譲してください。 "
                    "ユーザーのクエリを分析し、挨拶の場合は 'greeting_agent' に委譲し、別れの場合は 'farewell_agent' に委譲してください。 "
                    "天気リクエストの場合は、自分で 'get_weather' を使用して処理してください。 "
                    "その他のクエリについては、適切に応答するか、処理できないことを伝えてください。",
        tools=[get_weather], # ルートエージェントは依然として天気ツールが必要
        # 重要な変更: ここでサブエージェントをリンク！
        sub_agents=[greeting_agent, farewell_agent]
    )
    print(f"✅ ルートエージェント '{weather_agent_team.name}' がモデル '{root_agent_model}' を使用してサブエージェントと共に作成されました: {[sa.name for sa in weather_agent_team.sub_agents]}")

else:
    print("❌ ルートエージェントを作成できません。1つ以上のサブエージェントが初期化に失敗したか、'get_weather' ツールが欠落しています。")
    if not greeting_agent: print(" - 挨拶エージェントが欠落しています。")
    if not farewell_agent: print(" - 別れエージェントが欠落しています。")
    if 'get_weather' not in globals(): print(" - get_weather関数が欠落しています。")



---

**4\. エージェントチームと対話する**

ルートエージェント（`weather_agent_team` - *注: この変数名が前のコードブロックで定義されたものと一致していることを確認してください。おそらく`# @title Define the Root Agent with Sub-Agents`で`root_agent`と名付けられたものです*）とその専門サブエージェントを設定したので、委譲メカニズムをテストしましょう。

次のコードブロックは：

1. `async`関数`run_team_conversation`を定義します。
2. この関数内で、*新しい、専用の* `InMemorySessionService`と特定のセッション（`session_001_agent_team`）を作成し、このテスト実行専用にします。これにより、チームダイナミクスのテストのために会話履歴が分離されます。
3. `Runner`（`runner_agent_team`）を作成し、`weather_agent_team`（ルートエージェント）と専用のセッションサービスを使用するように設定します。
4. 更新された`call_agent_async`関数を使用して、異なるタイプのクエリ（挨拶、天気リクエスト、別れ）を`runner_agent_team`に送信します。特定のテストのためにランナー、ユーザーID、セッションIDを明示的に渡します。
5. `run_team_conversation`関数をすぐに実行します。

次のフローを期待します：

1. "Hello there!"クエリが`runner_agent_team`に送信されます。
2. ルートエージェント（`weather_agent_team`）が受信し、その指示と`greeting_agent`の説明に基づいてタスクを委譲します。
3. `greeting_agent`がクエリを処理し、`say_hello`ツールを呼び出し、応答を生成します。
4. "What is the weather in New York?"クエリは委譲されず、ルートエージェントが直接処理し、`get_weather`ツールを使用します。
5. "Thanks, bye!"クエリが`farewell_agent`に委譲され、`say_goodbye`ツールを使用します。

In [ ]:
# @title エージェントチームと対話する

# ルートエージェント（例: 'weather_agent_team' または 'root_agent'）が定義されていることを確認します。
# call_agent_async関数が定義されていることを確認します。

# ルートエージェント変数が存在するか確認してから会話関数を定義
root_agent_var_name = 'root_agent' # デフォルト名
if 'weather_agent_team' in globals(): # ユーザーがこの名前を使用したか確認
    root_agent_var_name = 'weather_agent_team'
elif 'root_agent' not in globals():
    print("⚠️ ルートエージェント（'root_agent' または 'weather_agent_team'）が見つかりません。run_team_conversationを定義できません。")
    # コードブロックが実行されてもNameErrorを防ぐためにダミー値を割り当て
    root_agent = None

if root_agent_var_name in globals() and globals()[root_agent_var_name]:
    async def run_team_conversation():
        print("\n--- エージェントチームの委譲をテスト ---")
        # InMemorySessionServiceは、このチュートリアルのためのシンプルで永続化されないストレージです。
        session_service = InMemorySessionService()

        # インタラクションコンテキストを識別するための定数を定義
        APP_NAME = "weather_tutorial_agent_team"
        USER_ID = "user_1_agent_team"
        SESSION_ID = "session_001_agent_team" # シンプルさのために固定IDを使用

        # 会話が行われる特定のセッションを作成
        session = session_service.create_session(
            app_name=APP_NAME,
            user_id=USER_ID,
            session_id=SESSION_ID
        )
        print(f"セッション作成: App='{APP_NAME}', User='{USER_ID}', Session='{SESSION_ID}'")

        # --- 実際のルートエージェントオブジェクトを取得 ---
        # 決定された変数名を使用
        actual_root_agent = globals()[root_agent_var_name]

        # このエージェントチームテストに特化したランナーを作成
        runner_agent_team = Runner(
            agent=actual_root_agent, # ルートエージェントオブジェクトを使用
            app_name=APP_NAME,       # 特定のアプリ名を使用
            session_service=session_service # 特定のセッションサービスを使用
            )
        # 実際のルートエージェントの名前を表示するための修正されたprint文
        print(f"ランナーがエージェント '{actual_root_agent.name}' のために作成されました。")

        # 常にルートエージェントのランナーを介して対話し、正しいIDを渡す
        await call_agent_async(query = "こんにちは！",
                               runner=runner_agent_team,
                               user_id=USER_ID,
                               session_id=SESSION_ID)
        await call_agent_async(query = "ニューヨークの天気はどうですか？",
                               runner=runner_agent_team,
                               user_id=USER_ID,
                               session_id=SESSION_ID)
        await call_agent_async(query = "ありがとう、さようなら！",
                               runner=runner_agent_team,
                               user_id=USER_ID,
                               session_id=SESSION_ID)

    # 会話を実行
    # 注: これはルートおよびサブエージェントが使用するモデルのAPIキーを必要とする場合があります！
    await run_team_conversation()
else:
    print("\n⚠️ エージェントチームの会話をスキップします。ルートエージェントが前のステップで正常に定義されませんでした。")


---

出力ログを注意深く確認し、特に`--- Tool: ... called ---`メッセージに注目してください。次のことが確認できます：

* "Hello there!"の場合、`say_hello`ツールが呼び出される（`greeting_agent`が処理したことを示す）。
* "What is the weather in New York?"の場合、`get_weather`ツールが呼び出される（ルートエージェントが処理したことを示す）。
* "Thanks, bye!"の場合、`say_goodbye`ツールが呼び出される（`farewell_agent`が処理したことを示す）。

これにより、**自動委譲**が成功したことが確認できます！ルートエージェントは、その指示とサブエージェントの`description`に基づいて、ユーザーリクエストを適切な専門エージェントに正しくルーティングしました。

これで、複数の協力エージェントを持つアプリケーションを構築するための基礎が整いました。このモジュール設計は、より複雑で能力の高いエージェントシステムを構築するための基本です。次のステップでは、エージェントがセッション状態を使用して情報を記憶し、パーソナライズする方法を学びます。

## ステップ 4: セッション状態を使用した記憶とパーソナライズの追加

これまで、エージェントチームは異なるタスクを委譲することができましたが、各インタラクションは新たに始まり、過去の会話やユーザーの好みを記憶していません。より高度でコンテキストに基づいた体験を提供するためには、エージェントに**記憶**が必要です。ADKは**セッション状態**を通じてこれを提供します。

**セッション状態とは？**

* 特定のユーザーセッション（`APP_NAME`、`USER_ID`、`SESSION_ID`で識別）に結びついたPython辞書（`session.state`）です。  
* セッション内の複数の会話ターンにわたって情報を保持します。  
* エージェントとツールはこの状態を読み書きでき、詳細を記憶し、行動を適応させ、応答をパーソナライズできます。

**エージェントが状態と対話する方法：**

1. **`ToolContext`（主な方法）:** ツールは`ToolContext`オブジェクトを受け入れることができ（最後の引数として宣言されている場合、ADKが自動的に提供）、これを通じて`tool_context.state`にアクセスし、実行中に好みを読み取ったり、結果を保存したりできます。  
2. **`output_key`（エージェント応答の自動保存）:** `Agent`は`output_key="your_key"`で設定でき、ADKはそのターンのエージェントの最終テキスト応答を`session.state["your_key"]`に自動的に保存します。

**このステップでは、Weather Botチームを次のように強化します：**

1. **新しい** `InMemorySessionService`を使用して、状態を分離してデモンストレーションします。  
2. ユーザーの`temperature_unit`の好みを定義してセッション状態を初期化します。  
3. この好みを`ToolContext`を介して読み取り、出力形式（摂氏/華氏）を調整する状態対応版の天気ツール（`get_weather_stateful`）を作成します。  
4. この状態対応ツールを使用するようにルートエージェントを更新し、`output_key`を設定して最終天気応答をセッション状態に自動保存します。  
5. 初期状態がツールにどのように影響するか、手動で状態を変更してその後の動作を変更する方法、`output_key`がエージェントの応答をどのように永続化するかを観察するために会話を実行します。

---

**1\. 新しいセッションサービスと状態を初期化**

状態管理を明確にデモンストレーションするために、`InMemorySessionService`の新しいインスタンスを作成します。また、ユーザーの好みの温度単位を定義してセッションを作成します。

In [ ]:
# @title 1. 新しいセッションサービスと状態を初期化

# 必要なセッションコンポーネントをインポート
from google.adk.sessions import InMemorySessionService

# この状態デモンストレーションのために新しいセッションサービスインスタンスを作成
session_service_stateful = InMemorySessionService()
print("✅ 状態デモンストレーションのために新しいInMemorySessionServiceが作成されました。")

# このチュートリアルのための新しいセッションIDを定義
SESSION_ID_STATEFUL = "session_state_demo_001"
USER_ID_STATEFUL = "user_state_demo"

# 初期状態データを定義 - ユーザーは最初に摂氏を好む
initial_state = {
    "user_preference_temperature_unit": "Celsius"
}

# 初期状態を提供してセッションを作成
session_stateful = session_service_stateful.create_session(
    app_name=APP_NAME, # 一貫したアプリ名を使用
    user_id=USER_ID_STATEFUL,
    session_id=SESSION_ID_STATEFUL,
    state=initial_state # <<< 作成時に状態を初期化
)
print(f"✅ ユーザー '{USER_ID_STATEFUL}' のためにセッション '{SESSION_ID_STATEFUL}' が作成されました。")

# 初期状態が正しく設定されたことを確認
retrieved_session = session_service_stateful.get_session(app_name=APP_NAME,
                                                         user_id=USER_ID_STATEFUL,
                                                         session_id = SESSION_ID_STATEFUL)
print("\n--- 初期セッション状態 ---")
if retrieved_session:
    print(retrieved_session.state)
else:
    print("エラー: セッションを取得できませんでした。")

---

**2\. 状態対応の天気ツール（`get_weather_stateful`）を作成**

次に、天気ツールの新しいバージョンを作成します。その主な機能は`ToolContext`を受け入れることで、`tool_context.state`にアクセスし、`user_preference_temperature_unit`を読み取り、温度を適切にフォーマットします。

* **キーコンセプト: `ToolContext`** このオブジェクトは、ツールロジックがセッションのコンテキストと対話し、状態変数を読み書きするためのブリッジです。最後のパラメータとして定義されている場合、ADKが自動的に注入します。

* **ベストプラクティス:** 状態を読み取る際には、キーが存在しない場合に備えて`dictionary.get('key', default_value)`を使用し、ツールがクラッシュしないようにします。

In [ ]:
from google.adk.tools.tool_context import ToolContext

def get_weather_stateful(city: str, tool_context: ToolContext) -> dict:
    """天気を取得し、セッション状態に基づいて温度単位を変換します。"""
    print(f"--- ツール: get_weather_stateful が呼び出されました。都市: {city} ---")

    # --- 状態から好みを読み取る ---
    preferred_unit = tool_context.state.get("user_preference_temperature_unit", "Celsius") # デフォルトは摂氏
    print(f"--- ツール: 状態 'user_preference_temperature_unit' を読み取る: {preferred_unit} ---")

    city_normalized = city.lower().replace(" ", "")

    # モック天気データ（内部的には常に摂氏で保存）
    mock_weather_db = {
        "newyork": {"temp_c": 25, "condition": "sunny"},
        "london": {"temp_c": 15, "condition": "cloudy"},
        "tokyo": {"temp_c": 18, "condition": "light rain"},
    }

    if city_normalized in mock_weather_db:
        data = mock_weather_db[city_normalized]
        temp_c = data["temp_c"]
        condition = data["condition"]

        # 状態の好みに基づいて温度をフォーマット
        if preferred_unit == "Fahrenheit":
            temp_value = (temp_c * 9/5) + 32 # 華氏を計算
            temp_unit = "°F"
        else: # デフォルトは摂氏
            temp_value = temp_c
            temp_unit = "°C"

        report = f"{city.capitalize()}の天気は{condition}で、気温は{temp_value:.0f}{temp_unit}です。"
        result = {"status": "success", "report": report}
        print(f"--- ツール: {preferred_unit}でレポートを生成。結果: {result} ---")

        # 状態に書き戻す例（このツールにはオプション）
        tool_context.state["last_city_checked_stateful"] = city
        print(f"--- ツール: 状態 'last_city_checked_stateful' を更新: {city} ---")

        return result
    else:
        # 都市が見つからない場合の処理
        error_msg = f"申し訳ありませんが、「{city}」の天気情報はありません。"
        print(f"--- ツール: 都市 '{city}' が見つかりませんでした。 ---")
        return {"status": "error", "error_message": error_msg}

print("✅ 状態対応の 'get_weather_stateful' ツールが定義されました。")


---

**3\. サブエージェントを再定義し、ルートエージェントを更新**

このステップが自己完結型で正しく構築されるように、まずステップ3で定義された`greeting_agent`と`farewell_agent`を再定義します。その後、新しいルートエージェント（`weather_agent_v4_stateful`）を定義します：

* 新しい`get_weather_stateful`ツールを使用します。  
* 挨拶と別れのサブエージェントを含めます。  
* **重要:** `output_key="last_weather_report"`を設定し、エージェントの最終天気応答をセッション状態に自動保存します。

In [ ]:
# @title 3. サブエージェントを再定義し、output_keyを持つルートエージェントを更新

# 必要なインポートを確認し、APIキーが設定されていることを確認（ステップ0/2から）
from google.adk.agents import Agent
from google.adk.models.lite_llm import LiteLlm
from google.adk.runners import Runner
# 'say_hello'、'say_goodbye'ツールが定義されていることを確認（ステップ3から）
# モデル定数MODEL_GPT_4O、MODEL_GEMINI_2_5_PROなどが定義されていることを確認

# --- 挨拶エージェントを再定義（ステップ3から） ---
greeting_agent = None
try:
    greeting_agent = Agent(
        model=MODEL_GEMINI_2_0_FLASH,
        name="greeting_agent",
        instruction="あなたは挨拶エージェントです。あなたの唯一のタスクは、ユーザーにフレンドリーな挨拶を提供することです。 "
                    "'say_hello'ツールを使用して挨拶を生成してください。 "
                    "他の会話やタスクには関与しないでください。",
        description="シンプルな挨拶とハローを 'say_hello' ツールを使用して処理します。",
        tools=[say_hello],
    )
    print(f"✅ エージェント '{greeting_agent.name}' が再定義されました。")
except Exception as e:
    print(f"❌ 挨拶エージェントを再定義できませんでした。エラー: {e}")

# --- 別れエージェントを再定義（ステップ3から） ---
farewell_agent = None
try:
    farewell_agent = Agent(
        model=MODEL_GEMINI_2_0_FLASH,
        name="farewell_agent",
        instruction="あなたは別れエージェントです。あなたの唯一のタスクは、丁寧な別れのメッセージを提供することです。 "
                    "'say_goodbye'ツールを使用してください。 "
                    "他のアクションは行わないでください。",
        description="シンプルな別れとさようならを 'say_goodbye' ツールを使用して処理します。",
        tools=[say_goodbye],
    )
    print(f"✅ エージェント '{farewell_agent.name}' が再定義されました。")
except Exception as e:
    print(f"❌ 別れエージェントを再定義できませんでした。エラー: {e}")

# --- 更新されたルートエージェントを定義 ---
root_agent_stateful = None
runner_root_stateful = None # ランナーを初期化

# ルートエージェントを作成する前にすべての前提条件を確認
if greeting_agent and farewell_agent and 'get_weather_stateful' in globals():

    root_agent_model = MODEL_GEMINI_2_0_FLASH # オーケストレーションモデルを選択

    root_agent_stateful = Agent(
        name="weather_agent_v4_stateful", # 新しいバージョン名
        model=root_agent_model,
        description="メインエージェント: 状態対応の単位で天気を提供し、挨拶/別れを委譲し、レポートを状態に保存します。",
        instruction="あなたはメインの天気エージェントです。'get_weather_stateful'を使用して天気を提供してください。 "
                    "ツールは状態に保存されたユーザーの好みに基づいて温度をフォーマットします。 "
                    "シンプルな挨拶は 'greeting_agent' に、別れは 'farewell_agent' に委譲してください。 "
                    "天気リクエスト、挨拶、別れのみを処理してください。",
        tools=[get_weather_stateful], # 状態対応ツールを使用
        sub_agents=[greeting_agent, farewell_agent], # サブエージェントを含める
        output_key="last_weather_report" # <<< エージェントの最終天気応答を自動保存
    )
    print(f"✅ 状態対応ツールとoutput_keyを使用してルートエージェント '{root_agent_stateful.name}' が作成されました。")

    # --- このルートエージェントと新しいセッションサービスのためのランナーを作成 ---
    runner_root_stateful = Runner(
        agent=root_agent_stateful,
        app_name=APP_NAME,
        session_service=session_service_stateful # 新しい状態対応セッションサービスを使用
    )
    print(f"✅ 状態対応ルートエージェント '{runner_root_stateful.agent.name}' のためのランナーが状態対応セッションサービスを使用して作成されました。")

else:
    print("❌ 状態対応ルートエージェントを作成できません。前提条件が欠落しているか、初期化に失敗しました。")
    if not greeting_agent: print(" - 挨拶エージェントが欠落しています。")
    if not farewell_agent: print(" - 別れエージェントが欠落しています。")
    if 'get_weather_stateful' not in globals(): print(" - 'get_weather_stateful' ツールが欠落しています。")


---

**4\. 状態フローをテストするために対話する**

次に、`runner_root_stateful`（状態対応エージェントと状態対応セッションサービスに関連付けられた）を使用して状態インタラクションをテストする会話を実行します。以前に定義した`call_agent_async`関数を使用し、正しいランナー、ユーザーID（`USER_ID_STATEFUL`）、セッションID（`SESSION_ID_STATEFUL`）を渡すことを確認します。

会話フローは次のようになります：

1. **天気を確認（ロンドン）:** `get_weather_stateful`ツールはセッション状態に初期化された「摂氏」好みを読み取り、温度をフォーマットします。ルートエージェントの最終応答（摂氏の天気レポート）は`output_key`設定を介して`state['last_weather_report']`に保存されます。
2. **状態を手動で更新:** `InMemorySessionService`インスタンス内に保存されている状態を*直接変更*します。
    * **なぜ直接変更するのか？** `session_service.get_session()`メソッドはセッションの*コピー*を返します。そのコピーを変更しても、後続のエージェント実行で使用される状態には影響しません。このテストシナリオでは、`InMemorySessionService`を使用して、内部の`sessions`辞書にアクセスし、`user_preference_temperature_unit`の実際の保存状態値を「華氏」に変更します。*注: 実際のアプリケーションでは、状態変更は通常、ツールやエージェントロジックが`EventActions(state_delta=...)`を返すことによってトリガーされ、手動の直接更新ではありません。*
3. **再度天気を確認（ニューヨーク）:** `get_weather_stateful`ツールは更新された「華氏」好みを状態から読み取り、温度を適切に変換します。ルートエージェントの*新しい*応答（華氏の天気）は`output_key`を介して前の値を上書きします。
4. **エージェントに挨拶:** 委譲が状態対応操作と共に正常に機能することを確認します。このインタラクションは、この特定のシーケンスで`output_key`によって保存される*最後の*応答になります。
5. **最終状態を確認:** 会話後にセッションを再度取得し（コピーを取得）、その状態を印刷して`user_preference_temperature_unit`が「華氏」であることを確認し、`output_key`によって保存された最終値（この実行では挨拶）を観察し、ツールによって書き込まれた`last_city_checked_stateful`値を確認します。

In [ ]:
# @title 4. 状態フローとoutput_keyをテストするために対話する

# 状態対応ランナー（runner_root_stateful）が前のセルから利用可能であることを確認
# call_agent_async、USER_ID_STATEFUL、SESSION_ID_STATEFUL、APP_NAMEが定義されていることを確認

if 'runner_root_stateful' in globals() and runner_root_stateful:
  async def run_stateful_conversation():
      print("\n--- 状態テスト: 温度単位の変換とoutput_key ---")

      # 1. 天気を確認（初期状態を使用: 摂氏）
      print("--- ターン1: ロンドンの天気をリクエスト（摂氏を期待） ---")
      await call_agent_async(query= "ロンドンの天気はどうですか？",
                             runner=runner_root_stateful,
                             user_id=USER_ID_STATEFUL,
                             session_id=SESSION_ID_STATEFUL
                            )

      # 2. 状態の好みを華氏に手動で更新 - 直接ストレージを変更
      print("\n--- 状態を手動で更新: 単位を華氏に設定 ---")
      try:
          # 内部ストレージに直接アクセス - これはテストのためのInMemorySessionServiceに特有
          stored_session = session_service_stateful.sessions[APP_NAME][USER_ID_STATEFUL][SESSION_ID_STATEFUL]
          stored_session.state["user_preference_temperature_unit"] = "Fahrenheit"
          # オプション: ロジックが依存する場合、タイムスタンプも更新することを検討
          # import time
          # stored_session.last_update_time = time.time()
          print(f"--- 保存されたセッション状態が更新されました。現在の 'user_preference_temperature_unit': {stored_session.state['user_preference_temperature_unit']} ---")
      except KeyError:
          print(f"--- エラー: ユーザー '{USER_ID_STATEFUL}' のアプリ '{APP_NAME}' 内のセッション '{SESSION_ID_STATEFUL}' を内部ストレージから取得できず、状態を更新できませんでした。IDとセッションが作成されたか確認してください。 ---")
      except Exception as e:
           print(f"--- 内部セッション状態の更新エラー: {e} ---")

      # 3. 再度天気を確認（ツールは現在華氏を使用するはず）
      # これにより、'last_weather_report'がoutput_keyを介して更新されます
      print("\n--- ターン2: ニューヨークの天気をリクエスト（華氏を期待） ---")
      await call_agent_async(query= "ニューヨークの天気を教えてください。",
                             runner=runner_root_stateful,
                             user_id=USER_ID_STATEFUL,
                             session_id=SESSION_ID_STATEFUL
                            )

      # 4. 基本的な委譲をテスト（依然として機能するはず）
      # これにより、'last_weather_report'が再度更新され、NYの天気レポートが上書きされます
      print("\n--- ターン3: 挨拶を送信 ---")
      await call_agent_async(query= "こんにちは！",
                             runner=runner_root_stateful,
                             user_id=USER_ID_STATEFUL,
                             session_id=SESSION_ID_STATEFUL
                            )

  # 会話を実行
  await run_stateful_conversation()

  # 会話後の最終セッション状態を確認
  print("\n--- 最終セッション状態を確認 ---")
  final_session = session_service_stateful.get_session(app_name=APP_NAME,
                                                       user_id=USER_ID_STATEFUL,
                                                       session_id=SESSION_ID_STATEFUL)
  if final_session:
      print(f"最終好み: {final_session.state.get('user_preference_temperature_unit')}")
      print(f"最終天気レポート（output_keyから）: {final_session.state.get('last_weather_report')}")
      print(f"ツールによる最終チェック都市: {final_session.state.get('last_city_checked_stateful')}")
      # 詳細ビューのために完全な状態を印刷
      # print(f"完全な状態: {final_session.state}")
  else:
      print("\n❌ エラー: 最終セッション状態を取得できませんでした。")

else:
  print("\n⚠️ 状態テスト会話をスキップします。状態対応ルートエージェントランナー（'runner_root_stateful'）が利用できません。")

---

会話フローと最終セッション状態の印刷を確認することで、次のことが確認できます：

* **状態読み取り:** 天気ツール（`get_weather_stateful`）が状態から`user_preference_temperature_unit`を正しく読み取り、最初にロンドンの天気を摂氏で提供しました。
* **状態更新:** 直接変更により、保存された好みが「華氏」に変更されました。
* **状態読み取り（更新済み）:** ツールがその後、状態から「華氏」を読み取り、温度を変換しました。
* **ツール状態書き込み:** ツールが`tool_context.state`を介して`last_city_checked_stateful`（2回目の天気チェック後の「ニューヨーク」）を状態に正常に書き込みました。
* **委譲:** "Hi!"の挨拶が状態変更後でも正常に`greeting_agent`に委譲されました。
* **`output_key`:** `output_key="last_weather_report"`が、ルートエージェントが最終的に応答した*各ターン*の最終応答を正常に保存しました。このシーケンスでは、最後の応答が挨拶（"Hello, there!"）であったため、状態キーの天気レポートが上書きされました。
* **最終状態:** 最終チェックにより、好みが「華氏」として永続化され、`output_key`によって保存された最終応答（この実行では挨拶）が確認され、ツールによって書き込まれた`last_city_checked_stateful`値が確認されました。

これで、`ToolContext`を使用してエージェントの動作をパーソナライズし、`InMemorySessionService`を使用して手動で状態を操作し、`output_key`がエージェントの最終応答を状態に保存する方法を理解しました。次のステップでは、コールバックを使用して安全ガードレールを実装します。

---

## ステップ 5: 安全性の追加 \- `before_model_callback`を使用した入力ガードレール

エージェントチームはますます能力を高め、好みを記憶し、ツールを効果的に使用できるようになりました。しかし、実際のシナリオでは、潜在的に問題のあるリクエストがコアの大規模言語モデル（LLM）に到達する*前に*制御するための安全メカニズムが必要です。

ADKは**コールバック**を提供します。これにより、エージェントの実行ライフサイクルの特定のポイントにフックすることができます。`before_model_callback`は入力の安全性に特に役立ちます。

**`before_model_callback`とは？**

* エージェントがコンパイルされたリクエスト（会話履歴、指示、最新のユーザーメッセージを含む）を基礎となるLLMに送信する*直前*にADKが実行するPython関数です。  
* **目的:** リクエストを検査し、必要に応じて変更するか、事前定義されたルールに基づいて完全にブロックします。

**一般的な使用例：**

* **入力検証/フィルタリング:** ユーザー入力が基準を満たしているか、許可されていないコンテンツ（PIIやキーワードなど）を含んでいないかを確認します。  
* **ガードレール:** 有害、オフトピック、ポリシー違反のリクエストがLLMによって処理されるのを防ぎます。  
* **動的プロンプト変更:** LLMリクエストコンテキストにタイムリーな情報（例: セッション状態から）を追加します。

**動作方法：**

1. `callback_context: CallbackContext`と`llm_request: LlmRequest`を受け入れる関数を定義します。  
   * `callback_context`: エージェント情報、セッション状態（`callback_context.state`）などにアクセスを提供します。  
   * `llm_request`: LLMに送信される予定の完全なペイロード（`contents`、`config`）を含みます。  
2. 関数内で：  
   * **検査:** `llm_request.contents`（特に最後のユーザーメッセージ）を調べます。  
   * **変更（注意して使用）:** `llm_request`の一部を変更できます。  
   * **ブロック（ガードレール）:** `LlmResponse`オブジェクトを返します。ADKはこの応答をすぐに返し、そのターンのLLM呼び出しを*スキップ*します。  
   * **許可:** `None`を返します。ADKは（潜在的に変更された）リクエストでLLMを呼び出します。

**このステップでは、以下を行います：**

1. ユーザー入力に特定のキーワード（"BLOCK"）が含まれているかをチェックする`before_model_callback`関数（`block_keyword_guardrail`）を定義します。
2. ステップ4の状態対応ルートエージェント（`weather_agent_v4_stateful`）を更新し、このコールバックを使用します。
3. 同じ状態対応セッションサービスを使用して、この更新されたエージェントの新しいランナーを作成します。
4. 通常のリクエストとキーワードを含むリクエストを送信してガードレールをテストします。

---

**1\. ガードレールコールバック関数を定義**

この関数は、`llm_request`コンテンツ内の最後のユーザーメッセージを検査します。"BLOCK"（大文字小文字を区別しない）を見つけた場合、`LlmResponse`を構築して返し、フローをブロックします。それ以外の場合は`None`を返します。

In [ ]:
# @title 1. before_model_callbackガードレールを定義

# 必要なインポートが利用可能であることを確認
from google.adk.agents.callback_context import CallbackContext
from google.adk.models.llm_request import LlmRequest
from google.adk.models.llm_response import LlmResponse
from google.genai import types # 応答コンテンツ作成用
from typing import Optional

def block_keyword_guardrail(
    callback_context: CallbackContext, llm_request: LlmRequest
) -> Optional[LlmResponse]:
    """
    最新のユーザーメッセージに 'BLOCK' が含まれているかを検査します。含まれている場合、
    LLM呼び出しをブロックし、事前定義されたLlmResponseを返します。それ以外の場合はNoneを返します。
    """
    agent_name = callback_context.agent_name # モデル呼び出しがインターセプトされるエージェントの名前を取得
    print(f"--- コールバック: block_keyword_guardrailがエージェント '{agent_name}' のために実行されています ---")

    # リクエスト履歴内の最新のユーザーメッセージからテキストを抽出
    last_user_message_text = ""
    if llm_request.contents:
        # 'user'ロールの最新のメッセージを見つける
        for content in reversed(llm_request.contents):
            if content.role == 'user' and content.parts:
                # 簡単のためにテキストが最初のパートにあると仮定
                if content.parts[0].text:
                    last_user_message_text = content.parts[0].text
                    break # 最新のユーザーメッセージテキストが見つかりました

    print(f"--- コールバック: 最新のユーザーメッセージを検査: '{last_user_message_text[:100]}...' ---") # 最初の100文字をログ

    # --- ガードレールロジック ---
    keyword_to_block = "BLOCK"
    if keyword_to_block in last_user_message_text.upper(): # 大文字小文字を区別しないチェック
        print(f"--- コールバック: '{keyword_to_block}' が見つかりました。LLM呼び出しをブロックします！ ---")
        # オプションで状態にフラグを設定してブロックイベントを記録
        callback_context.state["guardrail_block_keyword_triggered"] = True
        print(f"--- コールバック: 状態 'guardrail_block_keyword_triggered' を設定: True ---")

        # フローを停止し、これを代わりに返すためのLlmResponseを構築して返す
        return LlmResponse(
            content=types.Content(
                role="model", # エージェントの視点からの応答を模倣
                parts=[types.Part(text=f"このリクエストにはブロックされたキーワード '{keyword_to_block}' が含まれているため、処理できません。")],
            )
            # 注: 必要に応じてerror_messageフィールドを設定することもできます
        )
    else:
        # キーワードが見つからない場合、リクエストをLLMに進める
        print(f"--- コールバック: キーワードが見つかりませんでした。エージェント '{agent_name}' のLLM呼び出しを許可します。 ---")
        return None # Noneを返すと、ADKは通常通りに進行します

print("✅ block_keyword_guardrail関数が定義されました。")


---

**2\. ルートエージェントをコールバックを使用するように更新**

ルートエージェントを再定義し、`before_model_callback`パラメータを追加して新しいガードレール関数を指します。明確にするために新しいバージョン名を付けます。

*自己完結型実行ノート:* ステップ5と同様に、すべての前提条件（サブエージェント、ツール、`before_model_callback`）が定義または実行コンテキストで利用可能であることを確認してからこのエージェントを定義します。

In [ ]:
# @title 2. before_model_callbackを使用してルートエージェントを更新

# --- サブエージェントを再定義（このコンテキストで存在することを確認） ---
greeting_agent = None
try:
    # 定義されたモデル定数を使用
    greeting_agent = Agent(
        model=MODEL_GEMINI_2_0_FLASH,
        name="greeting_agent", # 一貫性のために元の名前を保持
        instruction="あなたは挨拶エージェントです。あなたの唯一のタスクは、ユーザーにフレンドリーな挨拶を提供することです。 "
                    "'say_hello'ツールを使用して挨拶を生成してください。他の会話やタスクには関与しないでください。",
        description="シンプルな挨拶とハローを 'say_hello' ツールを使用して処理します。",
        tools=[say_hello],
    )
    print(f"✅ サブエージェント '{greeting_agent.name}' が再定義されました。")
except Exception as e:
    print(f"❌ 挨拶エージェントを再定義できませんでした。エラー: {e}")

farewell_agent = None
try:
    # 定義されたモデル定数を使用
    farewell_agent = Agent(
        model=MODEL_GEMINI_2_0_FLASH,
        name="farewell_agent", # 元の名前を保持
        instruction="あなたは別れエージェントです。あなたの唯一のタスクは、丁寧な別れのメッセージを提供することです。 "
                    "'say_goodbye'ツールを使用してください。他のアクションは行わないでください。",
        description="シンプルな別れとさようならを 'say_goodbye' ツールを使用して処理します。",
        tools=[say_goodbye],
    )
    print(f"✅ サブエージェント '{farewell_agent.name}' が再定義されました。")
except Exception as e:
    print(f"❌ 別れエージェントを再定義できませんでした。エラー: {e}")


# --- コールバックを持つルートエージェントを定義 ---
root_agent_model_guardrail = None
runner_root_model_guardrail = None

# すべてのコンポーネントを確認してから進行
if greeting_agent and farewell_agent and 'get_weather_stateful' in globals() and 'block_keyword_guardrail' in globals():

    # 定義されたモデル定数を使用（例: MODEL_GEMINI_2_5_PRO）
    root_agent_model = MODEL_GEMINI_2_0_FLASH

    root_agent_model_guardrail = Agent(
        name="weather_agent_v5_model_guardrail", # 明確にするための新しいバージョン名
        model=root_agent_model,
        description="メインエージェント: 天気を処理し、挨拶/別れを委譲し、入力キーワードガードレールを含む。",
        instruction="あなたはメインの天気エージェントです。'get_weather_stateful'を使用して天気を提供してください。 "
                    "シンプルな挨拶は 'greeting_agent' に、別れは 'farewell_agent' に委譲してください。 "
                    "天気、挨拶、別れのみを処理してください。",
        tools=[get_weather],
        sub_agents=[greeting_agent, farewell_agent], # 再定義されたサブエージェントを参照
        output_key="last_weather_report", # ステップ4からoutput_keyを保持
        before_model_callback=block_keyword_guardrail # <<< ガードレールコールバックを割り当て
    )
    print(f"✅ ルートエージェント '{root_agent_model_guardrail.name}' がbefore_model_callbackを使用して作成されました。")

    # --- このエージェントのためのランナーを作成し、同じ状態対応セッションサービスを使用 ---
    # session_service_statefulがステップ4から存在することを確認
    if 'session_service_stateful' in globals():
        runner_root_model_guardrail = Runner(
            agent=root_agent_model_guardrail,
            app_name=APP_NAME, # 一貫したAPP_NAMEを使用
            session_service=session_service_stateful # <<< ステップ4のサービスを使用
        )
        print(f"✅ 状態対応セッションサービスを使用してガードレールエージェント '{runner_root_model_guardrail.agent.name}' のためのランナーが作成されました。")
    else:
        print("❌ ランナーを作成できません。ステップ4の 'session_service_stateful' が欠落しています。")

else:
    print("❌ モデルガードレールを持つルートエージェントを作成できません。1つ以上の前提条件が欠落しているか、初期化に失敗しました。")
    if not greeting_agent: print("   - 挨拶エージェント")
    if not farewell_agent: print("   - 別れエージェント")
    if 'get_weather_stateful' not in globals(): print("   - 'get_weather_stateful' ツール")
    if 'block_keyword_guardrail' not in globals(): print("   - 'block_keyword_guardrail' コールバック")

---

**3\. ガードレールをテストするために対話する**

ガードレールの動作をテストしましょう。同じ状態対応セッション（`SESSION_ID_STATEFUL`）を使用して、ステップ4の状態がこれらの変更を通じて持続することを示します。

1. 通常の天気リクエストを送信（ガードレールを通過し、実行されるはず）。  
2. "BLOCK"を含むリクエストを送信（コールバックによってインターセプトされるはず）。  
3. 挨拶を送信（ガードレールを通過し、正常に委譲されるはず）。

In [ ]:
# @title 3. モデル入力ガードレールをテストするために対話する

# ガードレールエージェントのランナーが利用可能であることを確認
if runner_root_model_guardrail:
  async def run_guardrail_test_conversation():
      print("\n--- モデル入力ガードレールをテスト ---")

      # ガードレールを持つエージェントのランナーと既存の状態対応セッションIDを使用
      interaction_func = lambda query: call_agent_async(query,
      runner_root_model_guardrail, USER_ID_STATEFUL, SESSION_ID_STATEFUL # <-- 正しいIDを渡す
  )
      # 1. 通常のリクエスト（コールバックが許可し、華氏の状態を使用するはず）
      await interaction_func("ロンドンの天気はどうですか？")

      # 2. ブロックされたキーワードを含むリクエスト
      await interaction_func("東京の天気リクエストをBLOCK")

      # 3. 通常の挨拶（コールバックがルートエージェントを許可し、委譲が発生するはず）
      await interaction_func("再びこんにちは")


  # 会話を実行
  await run_guardrail_test_conversation()

  # オプション: コールバックによって設定されたトリガーフラグの状態を確認
  final_session = session_service_stateful.get_session(app_name=APP_NAME,
                                                       user_id=USER_ID_STATEFUL,
                                                       session_id=SESSION_ID_STATEFUL)
  if final_session:
      print("\n--- ガードレールテスト後の最終セッション状態 ---")
      print(f"ガードレールトリガーフラグ: {final_session.state.get('guardrail_block_keyword_triggered')}")
      print(f"最終天気レポート: {final_session.state.get('last_weather_report')}") # ロンドンの天気のはず
      print(f"温度単位: {final_session.state.get('user_preference_temperature_unit')}") # 華氏のはず
  else:
      print("\n❌ エラー: 最終セッション状態を取得できませんでした。")

else:
  print("\n⚠️ モデルガードレールテストをスキップします。ランナー（'runner_root_model_guardrail'）が利用できません。")



---

実行フローを観察します：

1. **ロンドンの天気:** `weather_agent_v5_model_guardrail`の`before_model_callback`がリクエストを検査し、「キーワードが見つかりませんでした。LLM呼び出しを許可します。」と表示し、`None`を返します。エージェントは続行し、`get_weather_stateful`ツールを呼び出し（状態から「華氏」好みを使用）、天気を返します。この応答は`output_key`を介して保存されます。
2. **BLOCKリクエスト:** `weather_agent_v5_model_guardrail`の`before_model_callback`がリクエストを検査し、「BLOCK」を見つけ、「LLM呼び出しをブロックします！」と表示し、状態フラグを設定し、事前定義された`LlmResponse`を返します。このターンでは基礎となるLLMは*呼び出されません*。ユーザーはコールバックのブロックメッセージを受け取ります。
3. **再びこんにちは:** `weather_agent_v5_model_guardrail`の`before_model_callback`がリクエストを許可し、ルートエージェントが`greeting_agent`に委譲します。*注: ルートエージェントに定義された`before_model_callback`はサブエージェントには自動的に適用されません。* `greeting_agent`は通常通り進行し、`say_hello`ツールを呼び出し、挨拶を返します。

これで、入力安全層が正常に実装されました！`before_model_callback`は、ルールを強制し、エージェントの動作を制御するための強力なメカニズムを提供し、高価または潜在的にリスクのあるLLM呼び出しが行われる*前に*制御します。次に、ツールの使用自体にガードレールを追加するために同様の概念を適用します。

## ステップ 6: 安全性の追加 \- ツール引数ガードレール（`before_tool_callback`）

ステップ5では、ユーザー入力がLLMに到達する*前に*検査し、潜在的にブロックするガードレールを追加しました。次に、LLMがツールを使用することを決定した*後*、そのツールが実際に実行される*前*にもう一つの制御層を追加します。これは、LLMがツールに渡そうとしている*引数*を検証するのに役立ちます。

ADKはこの目的のために`before_tool_callback`を提供します。

**`before_tool_callback`とは？**

* LLMがツールの使用をリクエストし、引数を決定した後、特定のツール関数が実行される*直前*に実行されるPython関数です。  
* **目的:** ツール引数を検証し、特定の入力に基づいてツールの実行を防止し、動的に引数を変更し、リソース使用ポリシーを強制します。

**一般的な使用例：**

* **引数検証:** LLMが生成した引数が有効であるか、許可された範囲内であるか、期待される形式に準拠しているかを確認します。  
* **リソース保護:** 特定のパラメータでのAPI呼び出しをブロックするなど、コストがかかる、制限されたデータにアクセスする、望ましくない副作用を引き起こす可能性のある入力でツールが呼び出されるのを防ぎます。  
* **動的引数変更:** ツールが実行される前に、セッション状態や他のコンテキスト情報に基づいて引数を調整します。

**動作方法：**

1. `tool: BaseTool`、`args: Dict[str, Any]`、`tool_context: ToolContext`を受け入れる関数を定義します。  
   * `tool`: 呼び出されるツールオブジェクト（`tool.name`を検査）。  
   * `args`: ツールのためにLLMが生成した引数の辞書。  
   * `tool_context`: セッション状態（`tool_context.state`）、エージェント情報などにアクセスを提供します。  
2. 関数内で：  
   * **検査:** `tool.name`と`args`辞書を調べます。  
   * **変更:** `args`辞書内の値を*直接*変更します。`None`を返すと、ツールはこれらの変更された引数で実行されます。  
   * **ブロック/オーバーライド（ガードレール）:** **辞書**を返します。ADKはこの辞書をツール呼び出しの*結果*として扱い、元のツール関数の実行を完全に*スキップ*します。この辞書は、ブロックするツールの期待される戻り形式に一致する必要があります。  
   * **許可:** `None`を返します。ADKは（潜在的に変更された）引数で実際のツール関数を実行します。

**このステップでは、以下を行います：**

1. 特定のツール（`get_weather_stateful`）が「パリ」という都市で呼び出されたかをチェックする`before_tool_callback`関数（`block_paris_tool_guardrail`）を定義します。
2. 「パリ」が検出された場合、コールバックはツールをブロックし、カスタムエラーディクショナリを返します。
3. ルートエージェント（`weather_agent_v6_tool_guardrail`）を更新し、*両方*の`before_model_callback`と新しい`before_tool_callback`を含めます。
4. 同じ状態対応セッションサービスを使用して、このエージェントの新しいランナーを作成します。
5. 許可された都市とブロックされた都市（「パリ」）の天気をリクエストしてフローをテストします。

---

**1\. ツールガードレールコールバック関数を定義**

この関数は、`get_weather_stateful`ツールをターゲットにします。`city`引数をチェックします。「パリ」である場合、ツールをブロックし、ツールのエラーレスポンスのように見えるエラーディクショナリを返します。それ以外の場合は`None`を返してツールを実行します。

In [ ]:
# @title 1. before_tool_callbackガードレールを定義

# 必要なインポートが利用可能であることを確認
from google.adk.tools.base_tool import BaseTool
from google.adk.tools.tool_context import ToolContext
from typing import Optional, Dict, Any # 型ヒント用

def block_paris_tool_guardrail(
    tool: BaseTool, args: Dict[str, Any], tool_context: ToolContext
) -> Optional[Dict]:
    """
    'get_weather_stateful'が'パリ'のために呼び出されたかをチェックします。
    その場合、ツールの実行をブロックし、特定のエラーディクショナリを返します。
    それ以外の場合、ツール呼び出しを進行させるためにNoneを返します。
    """
    tool_name = tool.name
    agent_name = tool_context.agent_name # ツール呼び出しを試みるエージェント
    print(f"--- コールバック: block_paris_tool_guardrailがツール '{tool_name}' のためにエージェント '{agent_name}' で実行されています ---")
    print(f"--- コールバック: 引数を検査: {args} ---")

    # --- ガードレールロジック ---
    target_tool_name = "get_weather_stateful" # FunctionToolで使用される関数名に一致
    blocked_city = "paris"

    # 正しいツールであり、city引数がブロックされた都市に一致するか確認
    if tool_name == target_tool_name:
        city_argument = args.get("city", "") # 安全に'city'引数を取得
        if city_argument and city_argument.lower() == blocked_city:
            print(f"--- コールバック: ブロックされた都市 '{city_argument}' が検出されました。ツール実行をブロックします！ ---")
            # オプションで状態を更新
            tool_context.state["guardrail_tool_block_triggered"] = True
            print(f"--- コールバック: 状態 'guardrail_tool_block_triggered' を設定: True ---")

            # ツールの期待されるエラーフォーマットに一致するディクショナリを返す
            # このディクショナリがツールの結果となり、実際のツール実行がスキップされます。
            return {
                "status": "error",
                "error_message": f"ポリシー制限: '{city_argument.capitalize()}' の天気チェックは現在ツールガードレールによって無効化されています。"
            }
        else:
             print(f"--- コールバック: 都市 '{city_argument}' はツール '{tool_name}' に対して許可されています。 ---")
    else:
        print(f"--- コールバック: ツール '{tool_name}' はターゲットツールではありません。許可します。 ---")


    # 上記のチェックがディクショナリを返さなかった場合、ツールの実行を許可
    print(f"--- コールバック: ツール '{tool_name}' の実行を許可します。 ---")
    return None # Noneを返すと、実際のツール関数が実行されます

print("✅ block_paris_tool_guardrail関数が定義されました。")



---

**2\. 両方のコールバックを使用するようにルートエージェントを更新**

ルートエージェントを再定義し、`before_tool_callback`パラメータを追加して新しいガードレール関数を指します。明確にするために新しいバージョン名を付けます。

*自己完結型実行ノート:* ステップ5と同様に、すべての前提条件（サブエージェント、ツール、`before_model_callback`）が定義または実行コンテキストで利用可能であることを確認してからこのエージェントを定義します。

In [ ]:
# @title 両方のコールバックを持つルートエージェントを更新（自己完結型）

# --- 前提条件が定義されていることを確認 ---
# （定義または確認: Agent、LiteLlm、Runner、ToolContext、
#  モデル定数、say_hello、say_goodbye、greeting_agent、farewell_agent、
#  get_weather_stateful、block_keyword_guardrail、block_paris_tool_guardrail）

# --- サブエージェントを再定義（このコンテキストで存在することを確認） ---
greeting_agent = None
try:
    # 定義されたモデル定数を使用（例: MODEL_GPT_4O）
    greeting_agent = Agent(
        model=MODEL_GEMINI_2_0_FLASH,
        name="greeting_agent", # 一貫性のために元の名前を保持
        instruction="あなたは挨拶エージェントです。あなたの唯一のタスクは、ユーザーにフレンドリーな挨拶を提供することです。 "
                    "'say_hello'ツールを使用して挨拶を生成してください。他の会話やタスクには関与しないでください。",
        description="シンプルな挨拶とハローを 'say_hello' ツールを使用して処理します。",
        tools=[say_hello],
    )
    print(f"✅ サブエージェント '{greeting_agent.name}' が再定義されました。")
except Exception as e:
    print(f"❌ 挨拶エージェントを再定義できませんでした。モデル/APIキーを確認してください（{MODEL_GPT_4O}）。エラー: {e}")

farewell_agent = None
try:
    # 定義されたモデル定数を使用（例: MODEL_GPT_4O）
    farewell_agent = Agent(
        model=MODEL_GEMINI_2_0_FLASH,
        name="farewell_agent", # 元の名前を保持
        instruction="あなたは別れエージェントです。あなたの唯一のタスクは、丁寧な別れのメッセージを提供することです。 "
                    "'say_goodbye'ツールを使用してください。他のアクションは行わないでください。",
        description="シンプルな別れとさようならを 'say_goodbye' ツールを使用して処理します。",
        tools=[say_goodbye],
    )
    print(f"✅ サブエージェント '{farewell_agent.name}' が再定義されました。")
except Exception as e:
    print(f"❌ 別れエージェントを再定義できませんでした。モデル/APIキーを確認してください（{MODEL_GPT_4O}）。エラー: {e}")

# --- 両方のコールバックを持つルートエージェントを定義 ---
root_agent_tool_guardrail = None
runner_root_tool_guardrail = None

if ('greeting_agent' in globals() and greeting_agent and
    'farewell_agent' in globals() and farewell_agent and
    'get_weather_stateful' in globals() and
    'block_keyword_guardrail' in globals() and
    'block_paris_tool_guardrail' in globals()):

    root_agent_model = MODEL_GEMINI_2_0_FLASH

    root_agent_tool_guardrail = Agent(
        name="weather_agent_v6_tool_guardrail", # 新しいバージョン名
        model=root_agent_model,
        description="メインエージェント: 天気を処理し、委譲し、入力およびツールガードレールを含む。",
        instruction="あなたはメインの天気エージェントです。'get_weather_stateful'を使用して天気を提供してください。 "
                    "シンプルな挨拶は 'greeting_agent' に、別れは 'farewell_agent' に委譲してください。 "
                    "天気、挨拶、別れのみを処理してください。",
        tools=[get_weather_stateful],
        sub_agents=[greeting_agent, farewell_agent],
        output_key="last_weather_report",
        before_model_callback=block_keyword_guardrail, # モデルガードレールを保持
        before_tool_callback=block_paris_tool_guardrail # <<< ツールガードレールを追加
    )
    print(f"✅ 両方のコールバックを持つルートエージェント '{root_agent_tool_guardrail.name}' が作成されました。")

    # --- 同じ状態対応セッションサービスを使用してランナーを作成 ---
    if 'session_service_stateful' in globals():
        runner_root_tool_guardrail = Runner(
            agent=root_agent_tool_guardrail,
            app_name=APP_NAME,
            session_service=session_service_stateful # <<< ステップ4/5のサービスを使用
        )
        print(f"✅ 状態対応セッションサービスを使用してツールガードレールエージェント '{runner_root_tool_guardrail.agent.name}' のためのランナーが作成されました。")
    else:
        print("❌ ランナーを作成できません。ステップ4/5の 'session_service_stateful' が欠落しています。")

else:
    print("❌ ツールガードレールを持つルートエージェントを作成できません。前提条件が欠落しているか、初期化に失敗しました。")



---

**3\. ツールガードレールをテストするために対話する**

インタラクションフローをテストしましょう。同じ状態対応セッション（`SESSION_ID_STATEFUL`）を使用して、ステップ4の状態がこれらの変更を通じて持続することを示します。

1. "ニューヨーク"の天気をリクエスト：両方のコールバックを通過し、ツールが実行される（状態から華氏の好みを使用）。
2. "パリ"の天気をリクエスト：`before_model_callback`を通過。LLMが`get_weather_stateful(city='Paris')`をリクエスト。`before_tool_callback`がインターセプトし、ツールをブロックし、エラーディクショナリを返す。エージェントがこのエラーを中継。
3. "ロンドン"の天気をリクエスト：両方のコールバックを通過し、ツールが正常に実行される。

In [ ]:
# @title 3. ツール引数ガードレールをテストするために対話する

# ツールガードレールエージェントのランナーが利用可能であることを確認
if runner_root_tool_guardrail:
  async def run_tool_guardrail_test():
      print("\n--- ツール引数ガードレール（'パリ'がブロックされる）をテスト ---")

        # 両方のコールバックを持つエージェントのランナーと既存の状態対応セッションを使用
      interaction_func = lambda query: call_agent_async(query,
      runner_root_tool_guardrail, USER_ID_STATEFUL, SESSION_ID_STATEFUL
  )
      # 1. 許可された都市（両方のコールバックを通過し、華氏の状態を使用するはず）
      await interaction_func("ニューヨークの天気はどうですか？")

      # 2. ブロックされた都市（モデルコールバックを通過するが、ツールコールバックによってブロックされる）
      await interaction_func("パリはどうですか？")

      # 3. もう一つの許可された都市（再び正常に動作するはず）
      await interaction_func("ロンドンの天気を教えてください。")

  # 会話を実行
  await run_tool_guardrail_test()

  # オプション: ツールブロックトリガーフラグの状態を確認
  final_session = session_service_stateful.get_session(app_name=APP_NAME,
                                                       user_id=USER_ID_STATEFUL,
                                                       session_id= SESSION_ID_STATEFUL)
  if final_session:
      print("\n--- ツールガードレールテスト後の最終セッション状態 ---")
      print(f"ツールガードレールトリガーフラグ: {final_session.state.get('guardrail_tool_block_triggered')}")
      print(f"最終天気レポート: {final_session.state.get('last_weather_report')}") # ロンドンの天気のはず
      print(f"温度単位: {final_session.state.get('user_preference_temperature_unit')}") # 華氏のはず
  else:
      print("\n❌ エラー: 最終セッション状態を取得できませんでした。")

else:
  print("\n⚠️ ツールガードレールテストをスキップします。ランナー（'runner_root_tool_guardrail'）が利用できません。")

---

出力を分析します：

1. **ニューヨーク:** `before_model_callback`がリクエストを許可します。LLMが`get_weather_stateful`をリクエストします。`before_tool_callback`が実行され、引数（`{'city': 'New York'}`）を検査し、「パリ」ではないことを確認し、「ツールを許可します...」と表示して`None`を返します。実際の`get_weather_stateful`関数が実行され、状態から「華氏」を読み取り、天気レポートを返します。エージェントがこれを中継し、`output_key`を介して保存されます。
2. **パリ:** `before_model_callback`がリクエストを許可します。LLMが`get_weather_stateful(city='Paris')`をリクエストします。`before_tool_callback`が実行され、引数を検査し、「パリ」を検出し、「ツール実行をブロックします！」と表示し、状態フラグを設定し、エラーディクショナリ`{'status': 'error', 'error_message': 'ポリシー制限...'}`を返します。実際の`get_weather_stateful`関数は**実行されません**。エージェントはエラーディクショナリを*ツールの出力として*受け取り、そのエラーメッセージに基づいて応答を作成します。
3. **ロンドン:** ニューヨークと同様に動作し、両方のコールバックを通過し、ツールが正常に実行されます。新しいロンドンの天気レポートが状態の`last_weather_report`を上書きします。

これで、エージェントのツールの使用方法を制御するための重要な安全層が追加されました。`before_model_callback`と`before_tool_callback`のようなコールバックは、堅牢で安全でポリシーに準拠したエージェントアプリケーションを構築するために不可欠です。



---


## 結論: あなたのエージェントチームは準備完了です！

おめでとうございます！単一の基本的な天気エージェントの構築から、Agent Development Kit（ADK）を使用して高度なマルチエージェントチームを構築するまでの旅を成功させました。

**達成したことを振り返りましょう：**

* **基本的なエージェント**を構築し、単一のツール（`get_weather`）を装備しました。
* LiteLLMを使用してADKの**マルチモデルの柔軟性**を探り、異なるLLM（Gemini、GPT-4o、Claude）で同じコアロジックを実行しました。
* **モジュール性**を取り入れ、専門のサブエージェント（`greeting_agent`、`farewell_agent`）を作成し、**自動委譲**をルートエージェントから有効にしました。
* **セッション状態**を使用してエージェントに**記憶**を持たせ、ユーザーの好み（`temperature_unit`）や過去のインタラクション（`output_key`）を記憶させました。
* `before_model_callback`（特定の入力キーワードをブロック）と`before_tool_callback`（引数に基づいてツールの実行をブロック）を使用して重要な**安全ガードレール**を実装しました。

この進化したWeather Botチームを構築することで、複雑でインテリジェントなアプリケーションを開発するために必要なADKのコアコンセプトを実践的に学びました。

**重要なポイント：**

* **エージェントとツール:** 能力と推論を定義する基本的な構成要素。明確な指示とdocstringが重要です。
* **ランナーとセッションサービス:** エージェントの実行と会話のコンテキストを維持するエンジンとメモリ管理システム。
* **委譲:** マルチエージェントチームの設計により、専門化、モジュール性、複雑なタスクの管理が容易になります。エージェントの`description`が自動フローに重要です。
* **セッション状態（`ToolContext`、`output_key`）:** コンテキストに基づいたパーソナライズされたマルチターン会話エージェントを作成するために不可欠です。
* **コールバック（`before_model`、`before_tool`）:** 重要な操作（LLM呼び出しやツール実行）の*前に*安全性、検証、ポリシーの強制、動的変更を実装するための強力なフック。
* **柔軟性（`LiteLlm`）:** ADKは、パフォーマンス、コスト、機能のバランスを取りながら、最適なLLMを選択する力を提供します。

**次に進むべき場所は？**

Weather Botチームは素晴らしい出発点です。ADKをさらに探求し、アプリケーションを強化するためのアイデアをいくつか紹介します：

1. **実際の天気API:** `get_weather`ツールの`mock_weather_db`を実際の天気API（例: OpenWeatherMap、WeatherAPI）への呼び出しに置き換えます。
2. **より複雑な状態:** ユーザーの好み（例: 好みの場所、通知設定）や会話の要約をセッション状態に保存します。
3. **委譲の改善:** ルートエージェントの指示やサブエージェントの説明を実験して、委譲ロジックを微調整します。「予報」エージェントを追加できますか？
4. **高度なコールバック:**
    * `after_model_callback`を使用して、生成されたLLMの応答を再フォーマットまたはサニタイズします。
    * `after_tool_callback`を使用して、ツールが返す結果を処理またはログに記録します。
    * `before_agent_callback`や`after_agent_callback`を実装して、エージェントレベルのエントリ/エグジットロジックを追加します。
5. **エラーハンドリング:** ツールエラーや予期しないAPI応答の処理方法を改善します。ツール内でリトライロジックを追加することを検討してください。
6. **永続的なセッションストレージ:** セッション状態を永続的に保存するための`InMemorySessionService`の代替手段を探ります（例: FirestoreやCloud SQLなどのデータベースを使用 - カスタム実装または将来のADK統合が必要です）。
7. **ストリーミングUI:** Webフレームワーク（例: FastAPI、ADK Streaming Quickstartで示されているように）と統合して、リアルタイムチャットインターフェースを作成します。

Agent Development Kitは、洗練されたLLM駆動アプリケーションを構築するための堅牢な基盤を提供します。このチュートリアルでカバーしたコンセプト（ツール、状態、委譲、コールバック）をマスターすることで、ますます複雑なエージェントシステムに取り組む準備が整いました。

構築を楽しんでください！